In [2]:
# ============================================================
# 🎨 TEXT-TO-IMAGE GENERATOR USING OPENAI + GRADIO (LAB VERSION)
# ============================================================

# --- STEP 0: Install/align dependencies ---
#!pip install --upgrade openai gradio==4.44.1 huggingface_hub==0.22.2 Pillow httpx==0.27.2 --quiet

# --- Imports ---
import io, base64
from PIL import Image, ImageDraw
import gradio as gr
import openai  # ✅ Use the global client already configured in lab

# ============================================================
# STEP 1: Define image generation function (no manual API key)
# ============================================================
def generate_image(prompt: str):
    """
    Works with any OpenAI client variant found in training labs.
    Tries to extract a base64 image, URL, or string gracefully.
    """
    from PIL import Image, ImageDraw
    import io, base64, requests

    if not prompt or prompt.strip() == "":
        img = Image.new("RGB", (512, 512), color=(255, 230, 230))
        d = ImageDraw.Draw(img)
        d.text((20, 240), "⚠️ Please enter a valid prompt", fill=(0, 0, 0))
        return img

    try:
        print("🔍 Generating image for:", prompt)
        result = openai.Image.create(
            prompt=prompt,
            n=1,
            size="1024x1024"
        )

        # --- handle possible return shapes ---
        if isinstance(result, dict) and "data" in result:
            entry = result["data"][0]
            if "b64_json" in entry:
                image_bytes = base64.b64decode(entry["b64_json"])
                return Image.open(io.BytesIO(image_bytes))
            elif "url" in entry:
                image_bytes = requests.get(entry["url"]).content
                return Image.open(io.BytesIO(image_bytes))

        elif isinstance(result, str):
            # sometimes labs return a URL string directly
            if result.startswith("http"):
                image_bytes = requests.get(result).content
                return Image.open(io.BytesIO(image_bytes))
            raise ValueError(result)

        # fallback if structure unexpected
        raise ValueError(f"Unexpected API return type: {type(result)}")

    except Exception as e:
        err_msg = str(e)
        print(f"❌ API Error: {err_msg}")
        img = Image.new("RGB", (512, 512), color=(240, 240, 240))
        d = ImageDraw.Draw(img)
        d.text((20, 240), f"⚠️ API Error:\n{err_msg}", fill=(0, 0, 0))
        return img



# ============================================================
# STEP 2: Build and launch Gradio Interface
# ============================================================

demo = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(
        label="Enter your prompt",
        placeholder="e.g. A futuristic city skyline at sunset",
        lines=2
    ),
    outputs=gr.Image(label="Generated Image"),
    title="🧠 Text-to-Image Generator using OpenAI + Gradio",
    description="Enter a prompt to generate an AI-created image using OpenAI's DALL·E model. "
                "This version works automatically with the lab's built-in API key.",
    examples=[
        ["A cat wearing sunglasses"],
        ["A cyberpunk street at night, neon signs"],
        ["A fantasy castle floating in the clouds"]
    ]
)

demo.launch(share=True)


IMPORTANT: You are using gradio version 4.16.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://8dc3d24f60b62d93f3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


🔍 Generating image for: A cyberpunk street at night, neon signs
🔍 Generating image for: A cat wearing sunglasses
🔍 Generating image for: superman and batman fighting in the air
❌ API Error: Unexpected API return type: <class 'openai.openai_object.OpenAIObject'>
🔍 Generating image for: A fantasy castle floating in the clouds
❌ API Error: Unexpected API return type: <class 'openai.openai_object.OpenAIObject'>
